In [1]:
import os
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


/Users/yuantian/anaconda3/envs/9444/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/yuantian/anaconda3/envs/9444/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <2D1B8D5C-7891-3680-9CF9-F771AE880676> /Users/yuantian/anaconda3/envs/9444/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <E459C462-F863-3A5A-AC9F-FD77B14BE845> /Users/yuantian/anaconda3/envs/9444/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
DATA_DIR = "/Users/yuantian/Desktop/COMP9417 Machine Learning & Data Mining/assignment/project/Data - Needs Respray - 2024-03-26"
LABELS_FILE = "/Users/yuantian/Desktop/COMP9417 Machine Learning & Data Mining/assignment/project/Data - Needs Respray - 2024-03-26/Labels-NeedsRespray-2024-03-26.csv"


In [3]:
# 读取标签文件
labels_df = pd.read_csv(LABELS_FILE)

# 获取图像文件名和标签
filenames = labels_df['Filename']
labels = labels_df['Needs Respray']

# 定义训练集和测试集的划分比例
train_size = 0.6

# 划分训练集和测试集
train_filenames, test_filenames, train_labels, test_labels = train_test_split(filenames, labels, train_size=train_size, stratify=labels)


In [4]:
# 数据增强的转换
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整大小为ResNet的输入大小
    transforms.RandomHorizontalFlip(),  # 随机水平翻转
    transforms.ToTensor(),  # 将图像转换为张量
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # 标准化
])

# 自定义数据集类
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, filenames, labels, transform=None):
        self.data_dir = data_dir
        self.filenames = filenames
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_name = os.path.join(self.data_dir, self.filenames.iloc[idx])
        image = Image.open(img_name).convert('RGB')
        label = 1 if self.labels.iloc[idx] == 'Yes' else 0  # 将标签转换为二进制
        if self.transform:
            image = self.transform(image)
        return image, label

# 创建训练集和测试集的数据集实例
train_dataset = CustomDataset(DATA_DIR, train_filenames, train_labels, transform=transform)
test_dataset = CustomDataset(DATA_DIR, test_filenames, test_labels, transform=transform)

# 创建数据加载器
batch_size = 4
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [5]:
# 定义ResNet模型
resnet = torchvision.models.resnet18(pretrained=True)
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, 2)  # 二分类任务

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)

# 训练模型
num_epochs = 10
resnet.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

print('Finished Training')


/Users/yuantian/anaconda3/envs/9444/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/yuantian/anaconda3/envs/9444/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.6408951580524445
Epoch 2, Loss: 0.7019279301166534
Epoch 3, Loss: 0.6655472964048386
Epoch 4, Loss: 0.682316929101944
Epoch 5, Loss: 0.19026615470647812
Epoch 6, Loss: 0.44043418020009995
Epoch 7, Loss: 0.6319796741008759
Epoch 8, Loss: 0.19454752653837204
Epoch 9, Loss: 1.0738588869571686
Epoch 10, Loss: 0.26563214510679245
Finished Training


In [6]:
# 测试模型
resnet.eval()
predicted_labels = []
true_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = resnet(images)
        _, predicted = torch.max(outputs, 1)
        predicted_labels.extend(predicted.tolist())
        true_labels.extend(labels.tolist())

# 计算F1分数
f1 = f1_score(true_labels, predicted_labels)
print(f"F1 Score: {f1}")


F1 Score: 0.8


In [7]:
# 计算混淆矩阵
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[2 1]
 [0 2]]


In [8]:
import os
import pandas as pd
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
from PIL import Image
# 设置文件路径
data_dir = "/Users/yuantian/Desktop/COMP9417 Machine Learning & Data Mining/assignment/project/Data - Needs Respray - 2024-03-26"
label_file = "/Users/yuantian/Desktop/COMP9417 Machine Learning & Data Mining/assignment/project/Data - Needs Respray - 2024-03-26/Labels-NeedsRespray-2024-03-26.csv"

# 读取标签文件
labels_df = pd.read_csv(label_file)

# 构建图片路径
labels_df['Filepath'] = labels_df['Filename'].apply(lambda x: os.path.join(data_dir, x))

# 划分训练集和测试集
train_ratio = 0.8
train_size = int(len(labels_df) * train_ratio)
train_df = labels_df[:train_size]
test_df = labels_df[train_size:]

# 定义数据增强
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 定义自定义数据集类
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 2]  # 文件路径
        image = Image.open(img_name).convert('RGB')
        label = self.dataframe.iloc[idx, 1]  # 标签

        if self.transform:
            image = self.transform(image)

        # 将标签转换为张量
        label = torch.tensor(1 if label == 'Yes' else 0, dtype=torch.long)

        return image, label


# 创建训练集和测试集的数据加载器
train_dataset = CustomDataset(train_df, transform=transform)
test_dataset = CustomDataset(test_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# 定义VGG模型
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 2)  # 根据需要修改输出类别数量
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# 训练函数
def train_model(model, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# 测试函数
def test_model(model):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    f1 = f1_score(y_true, y_pred, average='weighted')
    print(f"F1 Score: {f1:.4f}")

# 使用GPU进行训练（如果可用）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 创建模型实例并移至GPU
model = VGG().to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
train_model(model, criterion, optimizer, num_epochs=10)

# 测试模型
test_model(model)


Epoch 1/10, Loss: 0.6536
Epoch 2/10, Loss: 0.9536
Epoch 3/10, Loss: 0.8373
Epoch 4/10, Loss: 0.9051
Epoch 5/10, Loss: 0.7043
Epoch 6/10, Loss: 0.6987
Epoch 7/10, Loss: 0.6873
Epoch 8/10, Loss: 0.6895
Epoch 9/10, Loss: 0.6978
Epoch 10/10, Loss: 0.6665
F1 Score: 0.1667
